In [ ]:
#!pip install tensorflow
#!pip install tqdm
#!pip install --upgrade music21
#!pip install pandas
#!pip install glob
#!pip install KERAS


In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [5]:
s = converter.parse('elise.mid')

In [6]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [7]:
n_vocab=len(set(notes))
print(n_vocab)

162


In [8]:
sequence_length = 500
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [9]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger-2MIDI.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=5, batch_size=64, callbacks=callbacks_list)

Epoch 1/5
5045/5045 [==============================] - ETA: 1253s - loss: 5.08 - ETA: 1395s - loss: 5.02 - ETA: 1461s - loss: 5.09 - ETA: 1533s - loss: 5.00 - ETA: 1579s - loss: 4.91 - ETA: 1647s - loss: 4.82 - ETA: 1689s - loss: 4.76 - ETA: 1706s - loss: 4.76 - ETA: 1731s - loss: 4.73 - ETA: 1752s - loss: 4.69 - ETA: 1789s - loss: 4.67 - ETA: 1819s - loss: 4.66 - ETA: 1838s - loss: 4.63 - ETA: 1877s - loss: 4.61 - ETA: 1902s - loss: 4.60 - ETA: 1961s - loss: 4.60 - ETA: 1998s - loss: 4.60 - ETA: 2024s - loss: 4.59 - ETA: 2050s - loss: 4.58 - ETA: 2088s - loss: 4.57 - ETA: 2123s - loss: 4.57 - ETA: 2147s - loss: 4.55 - ETA: 2161s - loss: 4.55 - ETA: 2171s - loss: 4.55 - ETA: 2166s - loss: 4.54 - ETA: 2176s - loss: 4.54 - ETA: 2180s - loss: 4.53 - ETA: 2178s - loss: 4.53 - ETA: 2165s - loss: 4.52 - ETA: 2154s - loss: 4.52 - ETA: 2132s - loss: 4.51 - ETA: 2115s - loss: 4.51 - ETA: 2097s - loss: 4.51 - ETA: 2076s - loss: 4.50 - ETA: 2049s - loss: 4.50 - ETA: 2026s - loss: 4.49 - ETA: 2008

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256)) 
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [7]:
# Load the weights to each node
model.load_weights('weights-improvement-05-3.8923-bigger-2MIDI.hdf5')

In [11]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(300):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [12]:
offset = 0.2
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [13]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='2midifileOutput.mid')

'2midifileOutput.mid'